In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import cv2
import numpy as np
import os
import glob
from sklearn.svm import LinearSVC
import time
from _datetime import datetime
import math
import re
import csv

In [ ]:
def teste():
    image= cv2.imread("C2.jpg");
    cv2.imshow("normal", image)
    
    image_ycrcb= cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)

    y,cr,cb= cv2.split(image_ycrcb)
    elementoEstruturante=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    fgmask=cv2.morphologyEx(y,cv2.MORPH_BLACKHAT,elementoEstruturante )

    fgmask= fgmask-y


    tipo= cv2.THRESH_BINARY+ cv2.THRESH_OTSU
    limiar,fgmask= cv2.threshold(fgmask,0,255,tipo)
    cv2.imshow("ex", fgmask)


    elementoEstruturante= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask= cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, elementoEstruturante)
    res_antes_canny= mask.copy()
    res= cv2.bitwise_and(image, image, mask=mask)
    
    cv2.imshow("ex1", mask)

    #Canny
    res= cv2.Canny(res, 100, 200)
    elementoEstruturante= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    imagemProcessada= cv2.dilate(res, elementoEstruturante, iterations= 2)
    imgTratada= cv2.morphologyEx(imagemProcessada, cv2.MORPH_CLOSE, elementoEstruturante)
    
    e= np.ones((3, 3), np.uint8)
    res=cv2.erode(imgTratada, e, iterations= 1)
    cv2.imshow("ex2", res)
    
    #inverter e mesclar com resultado antes de canny
    tipo= cv2.THRESH_BINARY_INV
    limiar,line= cv2.threshold(res,0,255,tipo)
    imagemProcessada= cv2.bitwise_and(res_antes_canny, res_antes_canny, mask=line)

    cv2.imshow("ex7", line)
    #retira todos os objetos menores que a mao, os preenche com preto (assumindo que o contorno da mao tenha maior area)
    _, contours, hierarchy = cv2.findContours(imagemProcessada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    for i in range(len(contours)):
        if(cv2.contourArea(cnt) > cv2.contourArea(contours[i])):
            cv2.drawContours(imagemProcessada, [contours[i]], 0, (0,0,0), cv2.FILLED)

    elementoEstruturante=cv2.getStructuringElement(	cv2.MORPH_ELLIPSE,(5,5) ) 
    imagemProcessada=cv2.morphologyEx(imagemProcessada,cv2.MORPH_CLOSE,elementoEstruturante )
    kernel = np.ones((3, 3), np.uint8)
    imagemProcessada=cv2.morphologyEx(imagemProcessada,cv2.MORPH_CLOSE,elementoEstruturante )
    imagemProcessada=cv2.morphologyEx(imagemProcessada,cv2.MORPH_CLOSE,elementoEstruturante )
    elementoEstruturante= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    imagemProcessada= cv2.dilate(imagemProcessada, elementoEstruturante, iterations= 1)
    imagemProcessada=cv2.morphologyEx(imagemProcessada,cv2.MORPH_CLOSE,elementoEstruturante )
 
    cv2.imshow("ex3", imagemProcessada)
    imagem_bin= imagemProcessada.copy()
        
    mask = np.zeros_like(imagem_bin)
    contour_hand= mask.copy()
    _, contours, hierarchy = cv2.findContours(imagem_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours:

        cv2.drawContours(mask,contours,-1,(255,255,255),1)
        copy_mask= dist=np.zeros((mask.shape[0],mask.shape[1]))
        for ind_y in range(mask.shape[0]):
            for ind_x in range(mask.shape[1]):
                copy_mask[ind_y,ind_x] = cv2.pointPolygonTest(contours[0],(ind_x,ind_y),True)

        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(copy_mask)
        centerMax= maxLoc
        ra= int(maxVal)
        #cv2.circle(mask,maxLoc,int(maxVal),(255, 255, 255),1)

        #circulo que contorna o objeto completamente utilizando a menor area possivel
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        centerMin = (int(x), int(y))
        rb = int(radius)



        import random as rng
        #criando uma mascara e desenhando o contorno da mao nela (isso porque nessa mascara da para po cor, no contorno da imagem binarizada nao da)
        mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
        for i in range(len(contours)):
            color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
            cv2.drawContours(mask, contours, i, color)
            #cv2.drawContours(drawing, hull_list, i, color)

        #convexidades
        hull = cv2.convexHull(contours[0],returnPoints = False)
        defects = cv2.convexityDefects(contours[0],hull)
        cnt= contours[0]
        defects_choosen= []
        pontos= []

        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem ### #VAI


            #b é a altura do triangulo de angulo 90
            if b>=ra and b<=rb and angle<=math.pi/2:

                #cv2.line(drawing,start,end,[0,255,0],2)
                #cv2.circle(drawing,far,5,[0,0,255],-1)
                if s not in defects_choosen:
                    defects_choosen.append(s)
                if e not in defects_choosen:
                    defects_choosen.append(e)
                    #pontos.append(start)
                    #pontos.append(end)

        defects_next= []
        for defect in defects_choosen:
            for d in defects_choosen:
                if defect != d:
                    if math.isclose(defect, d, rel_tol= 0.08, abs_tol=0.0):
                        defects_next.append(defect)
                        defects_choosen= diff_list(defects_choosen, defects_next)


        #calculando curvatura k
        if len(defects_choosen) > 0:
            for d in defects_choosen:
                #print(cnt[d])
                #for p in cnt:

                try:
                    ponto= tuple(cnt[d][0])
                    p1 = tuple(cnt[d+7])
                    p2 = tuple(cnt[d-7])

                    p1= tuple(p1[0])
                    p2= tuple(p2[0])

                    a = math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
                    b = math.sqrt((ponto[0] - p2[0]) ** 2 + (ponto[1] - p2[1]) ** 2)
                    c = math.sqrt((p1[0] - ponto[0]) ** 2 + (p1[1] - ponto[1]) ** 2)
                    angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem ### #VAI

                    '''a = math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
                    b = math.sqrt((ponto[0] - p1[0]) ** 2 + (ponto[1] - p1[1]) ** 2)
                    c = math.sqrt((p2[0] - ponto[0]) ** 2 + (p2[1] - ponto[1]) ** 2)
                    angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem ### #VAI
                    '''


                    if angle<=math.pi:
                        cv2.line(mask,p1,ponto,[0,255,0],2)
                        cv2.line(mask,p2,ponto,[0,255,0],2)
                        cv2.circle(mask,ponto,5,[0,0,255],-1)
                        #print("dedo")

                except:
                    print()

        else:
            distancia= 0
            for i in range(len(cnt)):
                #print(cnt[d])
                #for p in cnt:

                try:
                    if distancia == 0:	
                        ponto= tuple(cnt[i][0])
                        p1 = tuple(cnt[i+7])
                        p2 = tuple(cnt[i-7])

                        p1= tuple(p1[0])
                        p2= tuple(p2[0])

                        a = math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
                        b = math.sqrt((ponto[0] - p2[0]) ** 2 + (ponto[1] - p2[1]) ** 2)
                        c = math.sqrt((p1[0] - ponto[0]) ** 2 + (p1[1] - ponto[1]) ** 2)
                        angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem ### #VAI

                        '''a = math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
                        b = math.sqrt((ponto[0] - p1[0]) ** 2 + (ponto[1] - p1[1]) ** 2)
                        c = math.sqrt((p2[0] - ponto[0]) ** 2 + (p2[1] - ponto[1]) ** 2)
                        angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem ### #VAI
                        '''
                        if angle<=math.pi/3:
                            cv2.line(mask,p1,ponto,[0,255,0],2)
                            cv2.line(mask,p2,ponto,[0,255,0],2)
                            cv2.circle(mask,ponto,5,[0,0,255],-1)
                            #print("dedo")
                            distancia= 5
                    else:
                        distancia= distancia-1
                except:
                    print()

        cv2.circle(mask, centerMax, ra, (150, 20, 20), 3)
        cv2.circle(mask, centerMin, rb, (10, 20, 150), 3)



    cv2.imshow("imagem mask", mask)
    cv2.waitKey(0)
    

In [ ]:
teste()